<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# Practice Project

Estimated time needed: **60** minutes

This practice project focuses on data transformation and integration using PySpark. You will work with two datasets, perform various transformations such as adding columns, renaming columns, dropping unnecessary columns, joining dataframes, and finally, writing the results into both a Hive warehouse and an HDFS file system.


### Prerequisites 

For this lab assignment, you will use wget, Python and Spark (PySpark). Therefore, it's essential to make sure that the below-specified libraries are installed in your lab environment or within Skills Network (SN) Labs.  

 


In [2]:
# Installing required packages

!pip install wget pyspark  findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 MB 712.2 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 25.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.4-py2.py3-none-any.whl size=311905466 sha256=0f8c4cd1bac6da678a8c8e5b40e85856d0ea571c8c8258dff61e022a8eed2e17
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/4e/66/db/939eb1c49afb8a7fd2c4e393ad34e12b77db67bb4cc974c00e
Successfully built pyspark


#### Prework - Initiate the Spark Session


In [3]:
import findspark

findspark.init()

In [4]:
# PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the SparkContext.   

from pyspark import SparkContext, SparkConf

from pyspark.sql import SparkSession

In [5]:
# Creating a SparkContext object

sc = SparkContext.getOrCreate()

# Creating a Spark Session

spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

25/12/26 05:01:41 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Task 1: Load datasets into PySpark DataFrames

Download the datasets from the folloing links using `wget` and load it in a Spark Dataframe.

1. https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset1.csv  
2. https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset2.csv  

*Hint: Import wget*


In [6]:
#download dataset using wget
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset1.csv
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset2.csv

--2025-12-26 05:01:51--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset1.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4115 (4.0K) [text/csv]
Saving to: ‘dataset1.csv.1’

dataset1.csv.1      100%[===================>]   4.02K  --.-KB/s    in 0s      

2025-12-26 05:01:51 (31.3 MB/s) - ‘dataset1.csv.1’ saved [4115/4115]

--2025-12-26 05:01:52--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset2.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.clo


<details>
    <summary>Click here for Solution</summary>

```python
# download the dataset using wget
import wget

link_to_data1 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset1.csv'
wget.download(link_to_data1)

link_to_data2 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset2.csv'
wget.download(link_to_data2)
```

</details>


In [7]:
#load the data into a pyspark dataframe

# Load the datasets
df1 = spark.read.csv("dataset1.csv", header=True, inferSchema=True)
df2 = spark.read.csv("dataset2.csv", header=True, inferSchema=True)

# Display a few rows
df1.show(5)
df2.show(5)

+-----------+-----------+------+-----------+--------+
|customer_id|date_column|amount|description|location|
+-----------+-----------+------+-----------+--------+
|          1|   1/1/2022|  5000| Purchase A| Store A|
|          2|  15/2/2022|  1200| Purchase B| Store B|
|          3|  20/3/2022|   800| Purchase C| Store C|
|          4|  10/4/2022|  3000| Purchase D| Store D|
|          5|   5/5/2022|  6000| Purchase E| Store E|
+-----------+-----------+------+-----------+--------+
only showing top 5 rows

+-----------+----------------+-----+------+
|customer_id|transaction_date|value| notes|
+-----------+----------------+-----+------+
|          1|        1/1/2022| 1500|Note 1|
|          2|       15/2/2022| 2000|Note 2|
|          3|       20/3/2022| 1000|Note 3|
|          4|       10/4/2022| 2500|Note 4|
|          5|        5/5/2022| 1800|Note 5|
+-----------+----------------+-----+------+
only showing top 5 rows




<details>
    <summary>Click here for Solution</summary>

```python

#load the data into a pyspark dataframe
    
df1 = spark.read.csv("dataset1.csv", header=True, inferSchema=True)
df2 = spark.read.csv("dataset2.csv", header=True, inferSchema=True)
```

</details>




### Task 2: Display the schema of both dataframes

Display the schema of `df1` and `df2` to understand the structure of the datasets.


In [8]:
#print the schema of df1 and df2

df1.printSchema()
df2.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- date_column: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- description: string (nullable = true)
 |-- location: string (nullable = true)

root
 |-- customer_id: integer (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- value: integer (nullable = true)
 |-- notes: string (nullable = true)




<details>
    <summary>Click here for Solution</summary>

```python

#print the schema of df1 and df2
    
df1.printSchema()
df2.printSchema()
```

</details>




#### Task 3: Add a new column to each dataframe

Add a new column named **year** to `df1` and **quarter** to `df2` representing the year and quarter of the data.

*Hint: use withColumn. Convert the date columns which are present as string to date before extracting the year and quarter information*




In [9]:
from pyspark.sql.functions import year, quarter, to_date

#Add new column year to df1
# df1 = df1.withColumn("year", year(df1["date"]))  # if already in date type
df1 = df1.withColumn('year', year(to_date('date_column','dd/MM/yyyy')))

#Add new column quarter to df2    

# df2 = df2.withColumn("quarter", quarter(df2["date"]))  # if already in date type
df2 = df2.withColumn('quarter', quarter(to_date('transaction_date','dd/MM/yyyy')))

# Show results
df1.show(5)
df2.show(5)


"""
Best Practice approach
# Convert string → date
df1 = df1.withColumn("date", to_date("date_column", "dd/MM/yyyy"))
df2 = df2.withColumn("date", to_date("transaction_date", "dd/MM/yyyy"))

# Extract fields
df1 = df1.withColumn("year", year("date"))
df2 = df2.withColumn("quarter", quarter("date"))

"""

+-----------+-----------+------+-----------+--------+----+
|customer_id|date_column|amount|description|location|year|
+-----------+-----------+------+-----------+--------+----+
|          1|   1/1/2022|  5000| Purchase A| Store A|2022|
|          2|  15/2/2022|  1200| Purchase B| Store B|2022|
|          3|  20/3/2022|   800| Purchase C| Store C|2022|
|          4|  10/4/2022|  3000| Purchase D| Store D|2022|
|          5|   5/5/2022|  6000| Purchase E| Store E|2022|
+-----------+-----------+------+-----------+--------+----+
only showing top 5 rows

+-----------+----------------+-----+------+-------+
|customer_id|transaction_date|value| notes|quarter|
+-----------+----------------+-----+------+-------+
|          1|        1/1/2022| 1500|Note 1|      1|
|          2|       15/2/2022| 2000|Note 2|      1|
|          3|       20/3/2022| 1000|Note 3|      1|
|          4|       10/4/2022| 2500|Note 4|      2|
|          5|        5/5/2022| 1800|Note 5|      2|
+-----------+---------------

'\nBest Practice approach\n# Convert string → date\ndf1 = df1.withColumn("date", to_date("date_column", "dd/MM/yyyy"))\ndf2 = df2.withColumn("date", to_date("transaction_date", "dd/MM/yyyy"))\n\n# Extract fields\ndf1 = df1.withColumn("year", year("date"))\ndf2 = df2.withColumn("quarter", quarter("date"))\n\n'


<details>
    <summary>Click here for Solution</summary>

```python
from pyspark.sql.functions import year, quarter

#Add new column year to df1
df1 = df1.withColumn('year', year(to_date('date_column','dd/MM/yyyy')))
    
#Add new column quarter to df2    
df2 = df2.withColumn('quarter', quarter(to_date('transaction_date','dd/MM/yyyy')))```

</details>





#### Task 4: Rename columns in both dataframes

Rename the column **amount** to **transaction_amount** in `df1` and **value** to **transaction_value** in `df2`.

*Hint: Use withColumnRenamed*


In [10]:
#Rename df1 column amount to transaction_amount
df1 = df1.withColumnRenamed("amount", "transaction_amount")

#Rename df2 column value to transaction_value
df2 = df2.withColumnRenamed("value", "transaction_value")

# Show results
df1.show(5)
df2.show(5)

df1.printSchema()
df2.printSchema()

+-----------+-----------+------------------+-----------+--------+----+
|customer_id|date_column|transaction_amount|description|location|year|
+-----------+-----------+------------------+-----------+--------+----+
|          1|   1/1/2022|              5000| Purchase A| Store A|2022|
|          2|  15/2/2022|              1200| Purchase B| Store B|2022|
|          3|  20/3/2022|               800| Purchase C| Store C|2022|
|          4|  10/4/2022|              3000| Purchase D| Store D|2022|
|          5|   5/5/2022|              6000| Purchase E| Store E|2022|
+-----------+-----------+------------------+-----------+--------+----+
only showing top 5 rows

+-----------+----------------+-----------------+------+-------+
|customer_id|transaction_date|transaction_value| notes|quarter|
+-----------+----------------+-----------------+------+-------+
|          1|        1/1/2022|             1500|Note 1|      1|
|          2|       15/2/2022|             2000|Note 2|      1|
|          3|   



<details>
    <summary>Click here for Solution</summary>

```python

#Rename df1 column amount to transaction_amount
df1 = df1.withColumnRenamed('amount', 'transaction_amount')
    
#Rename df2 column value to transaction_value
df2 = df2.withColumnRenamed('value', 'transaction_value')
```

</details>





#### Task 5: Drop unnecessary columns

Drop the columns **description** and **location** from `df1` and **notes** from `df2`.




In [11]:
#Drop columns description and location from df1
df1 = df1.drop("description", "location")

#Drop column notes from df2
df2 = df2.drop("notes")

# Show results
df1.show(5)
df2.show(5)

df1.printSchema()
df2.printSchema()

+-----------+-----------+------------------+----+
|customer_id|date_column|transaction_amount|year|
+-----------+-----------+------------------+----+
|          1|   1/1/2022|              5000|2022|
|          2|  15/2/2022|              1200|2022|
|          3|  20/3/2022|               800|2022|
|          4|  10/4/2022|              3000|2022|
|          5|   5/5/2022|              6000|2022|
+-----------+-----------+------------------+----+
only showing top 5 rows

+-----------+----------------+-----------------+-------+
|customer_id|transaction_date|transaction_value|quarter|
+-----------+----------------+-----------------+-------+
|          1|        1/1/2022|             1500|      1|
|          2|       15/2/2022|             2000|      1|
|          3|       20/3/2022|             1000|      1|
|          4|       10/4/2022|             2500|      2|
|          5|        5/5/2022|             1800|      2|
+-----------+----------------+-----------------+-------+
only showing




<details>
    <summary>Click here for Solution</summary>

```python

#Drop columns description and location from df1
df1 = df1.drop('description', 'location')
    
#Drop column notes from df2
df2 = df2.drop('notes')
```

</details>





#### Task 6: Join dataframes based on a common column

Join `df1` and `df2` based on the common column **customer_id** and create a new dataframe named `joined_df`.




In [14]:
#join df1 and df2 based on common column customer_id
joined_df = df1.join(df2, on="customer_id", how="inner")
joined_df.show(10)
joined_df.printSchema()


+-----------+-----------+------------------+----+----------------+-----------------+-------+
|customer_id|date_column|transaction_amount|year|transaction_date|transaction_value|quarter|
+-----------+-----------+------------------+----+----------------+-----------------+-------+
|          1|   1/1/2022|              5000|2022|        1/1/2022|             1500|      1|
|          2|  15/2/2022|              1200|2022|       15/2/2022|             2000|      1|
|          3|  20/3/2022|               800|2022|       20/3/2022|             1000|      1|
|          4|  10/4/2022|              3000|2022|       10/4/2022|             2500|      2|
|          5|   5/5/2022|              6000|2022|        5/5/2022|             1800|      2|
|          6|  10/6/2022|              4500|2022|       10/6/2022|             1200|      2|
|          7|  15/7/2022|               200|2022|       15/7/2022|              700|      3|
|          8|  20/8/2022|              3500|2022|       20/8/2022|    

<details>
    <summary>Click here for Solution</summary>

```python

#join df1 and df2 based on common column customer_id
joined_df = df1.join(df2, 'customer_id', 'inner')
    
```

</details>


#### Task 7: Filter data based on a condition

Filter `joined_df` to include only transactions where "transaction_amount" is greater than 1000 and create a new dataframe named `filtered_df`.





In [15]:
# filter the dataframe for transaction amount > 1000
filtered_df = joined_df.filter(joined_df.transaction_amount > 1000)
filtered_df.show(10)

+-----------+-----------+------------------+----+----------------+-----------------+-------+
|customer_id|date_column|transaction_amount|year|transaction_date|transaction_value|quarter|
+-----------+-----------+------------------+----+----------------+-----------------+-------+
|          1|   1/1/2022|              5000|2022|        1/1/2022|             1500|      1|
|          2|  15/2/2022|              1200|2022|       15/2/2022|             2000|      1|
|          4|  10/4/2022|              3000|2022|       10/4/2022|             2500|      2|
|          5|   5/5/2022|              6000|2022|        5/5/2022|             1800|      2|
|          6|  10/6/2022|              4500|2022|       10/6/2022|             1200|      2|
|          8|  20/8/2022|              3500|2022|       20/8/2022|             3000|      3|
|         10| 30/10/2022|              1800|2022|      30/10/2022|             1200|      4|
|         11|  5/11/2022|              2200|2022|       5/11/2022|    

<details>
    <summary>Click here for Solution</summary>

```python

# filter the dataframe for transaction amount > 1000
filtered_df = joined_df.filter("transaction_amount > 1000")    
```

</details>


#### Task 8: Aggregate data by customer

Calculate the total transaction amount for each customer in `filtered_df` and display the result.

*Hint: Use sum from pyspark.sql.functions*


In [18]:

# group by customer_id and aggregate the sum of transaction amount
from pyspark.sql.functions import sum

total_transaction_eachCustomer_df = filtered_df.groupBy("customer_id") \
                       .agg(sum("transaction_amount").alias("total_transaction_amount"))


#display the result
total_transaction_eachCustomer_df.show(5)


+-----------+------------------------+
|customer_id|total_transaction_amount|
+-----------+------------------------+
|         31|                    3200|
|         85|                    1800|
|         78|                    1500|
|         34|                    1200|
|         81|                    5500|
+-----------+------------------------+
only showing top 5 rows



<details>
    <summary>Click here for Solution</summary>

```python

from pyspark.sql.functions import sum
   
# group by customer_id and aggregate the sum of transaction amount

total_amount_per_customer = filtered_df.groupBy('customer_id').agg(sum('transaction_amount').alias('total_amount'))

#display the result
total_amount_per_customer.show()
```

</details>


#### Task 9: Write the result to a Hive table

Write `total_amount_per_customer` to a Hive table named **customer_totals**.


In [22]:
# Write total_amount_per_customer to a Hive table named customer_totals
total_amount_per_customer = total_transaction_eachCustomer_df
total_amount_per_customer.write.mode("overwrite").saveAsTable("customer_totals")
spark.sql("SELECT * FROM customer_totals").show()
# spark.sql("SHOW TABLES").show()
# spark.sql("SELECT * FROM customer_totals").exceptAll(total_transaction_eachCustomer_df).count()

# OR
total_amount_per_customer.write.mode("overwrite").saveAsTable("analytics.customer_totals")
spark.sql("SELECT * FROM analytics.customer_totals").show()

[Stage 52:>                                                         (0 + 1) / 1]

+-----------+------------------------+
|customer_id|total_transaction_amount|
+-----------+------------------------+
|         25|                    1800|
|         73|                    1800|
|          6|                    4500|
|         16|                    2600|
|         15|                    4200|
|         43|                    3200|
|          4|                    3000|
|         55|                    3200|
|          8|                    3500|
|        100|                    2600|
|         60|                    2400|
|         90|                    1500|
|         11|                    2200|
|         33|                    5500|
|          2|                    1200|
|         79|                    3200|
|         30|                    1500|
|         99|                    4200|
|         46|                    1200|
|         67|                    3200|
+-----------+------------------------+
only showing top 20 rows




<details>
    <summary>Click here for Solution</summary>

```python

# Write total_amount_per_customer to a Hive table named customer_totals
total_amount_per_customer.write.mode("overwrite").saveAsTable("customer_totals")
```

</details>


#### Task 10: Write the filtered data to HDFS

Write `filtered_df` to HDFS in parquet format to a file named **filtered_data**.


In [23]:
#Write filtered_df to HDFS in parquet format file filtered_data.parquet
# filtered_df.write.mode("overwrite").parquet("hdfs://namenode_host:port/path/to/filtered_data")
# # filtered_df.write.mode("overwrite").parquet("hdfs://localhost:9000/user/hive/warehouse/filtered_data")
filtered_df.write.mode("overwrite").parquet("filtered_data.parquet")



<details>
    <summary>Click here for Solution</summary>

```python

#Write filtered_df to HDFS in parquet format file filtered_data

filtered_df.write.mode("overwrite").parquet("filtered_data.parquet")
```

</details>


#### Task 11: Add a new column based on a condition

Add a new column named **high_value** to `df1` indicating whether the transaction_amount is greater than 5000. When the value is greater than 5000, the value of the column should be **Yes**. When the value is less than or equal to 5000, the value of the column should be **No**. 

*Hint: Use when and lit from pyspark.sql.functions


In [25]:
# Add new column with value indicating whether transaction amount is > 5000 or not
from pyspark.sql.functions import when, lit, col

df1 = df1.withColumn(
    "high_value",
    when(col("transaction_amount") > 5000, lit("Yes"))
    .otherwise(lit("No"))
)
df1.show(5)
# from pyspark.sql.functions import when, col

# df1 = df1.withColumn(
#     "high_value",
#     when(col("transaction_amount") > 5000, "Yes").otherwise("No")
# )


+-----------+-----------+------------------+----+----------+
|customer_id|date_column|transaction_amount|year|high_value|
+-----------+-----------+------------------+----+----------+
|          1|   1/1/2022|              5000|2022|        No|
|          2|  15/2/2022|              1200|2022|        No|
|          3|  20/3/2022|               800|2022|        No|
|          4|  10/4/2022|              3000|2022|        No|
|          5|   5/5/2022|              6000|2022|       Yes|
+-----------+-----------+------------------+----+----------+
only showing top 5 rows




<details>
    <summary>Click here for Solution</summary>

```python

from pyspark.sql.functions import when, lit

# Add new column with value indicating whether transaction amount is > 5000 or not
df1 = df1.withColumn("high_value", when(df1.transaction_amount > 5000, lit("Yes")).otherwise(lit("No")))
```

</details>


#### Task 12: Calculate the average transaction value per quarter

Calculate and display the average transaction value for each quarter in `df2` and create a new dataframe named `average_value_per_quarter` with column `avg_trans_val`.

*Hint: Use avg from pyspark.sql.functions*


In [26]:
#calculate the average transaction value for each quarter in df2
from pyspark.sql.functions import avg

average_value_per_quarter = (
    df2.groupBy("quarter")
       .agg(avg("transaction_value").alias("avg_trans_val"))
)

#show the average transaction value for each quarter in df2    
average_value_per_quarter.show()


+-------+------------------+
|quarter|     avg_trans_val|
+-------+------------------+
|      1| 1111.111111111111|
|      3|1958.3333333333333|
|      4| 816.6666666666666|
|      2|            1072.0|
+-------+------------------+




<details>
    <summary>Click here for Solution</summary>

```python
from pyspark.sql.functions import avg

#calculate the average transaction value for each quarter in df2
average_value_per_quarter = df2.groupBy('quarter').agg(avg("transaction_value").alias("avg_trans_val"))

    
#show the average transaction value for each quarter in df2    
average_value_per_quarter.show()

```

</details>


#### Task 13: Write the result to a Hive table

Write `average_value_per_quarter` to a Hive table named **quarterly_averages**.


In [27]:
#Write average_value_per_quarter to a Hive table named quarterly_averages
average_value_per_quarter.write.mode("overwrite").saveAsTable("quarterly_averages")
spark.sql("SELECT * FROM quarterly_averages").show()


+-------+------------------+
|quarter|     avg_trans_val|
+-------+------------------+
|      1| 1111.111111111111|
|      3|1958.3333333333333|
|      4| 816.6666666666666|
|      2|            1072.0|
+-------+------------------+




<details>
    <summary>Click here for Solution</summary>

```python

#Write average_value_per_quarter to a Hive table named quarterly_averages

average_value_per_quarter.write.mode("overwrite").saveAsTable("quarterly_averages")

```

</details>


#### Task 14: Calculate the total transaction value per year

Calculate and display the total transaction value for each year in `df1` and create a new dataframe named `total_value_per_year` with column `total_transaction_val`.
> **Note:** The provided DataFrame `df1` does not explicitly have a `year` column initially. However, in Task 3, a new column named `year` is added to `df1` by extracting the year from the date column. Additionally, in Task 4, the column `amount` is renamed to `transaction_amount`.



In [28]:
# calculate the total transaction value for each year in df1.
# from pyspark.sql.functions import sum

total_value_per_year = (
    df1.groupBy("year")
       .agg(sum("transaction_amount").alias("total_transaction_val"))
)

# show the total transaction value for each year in df1.
total_value_per_year.show()


+----+---------------------+
|year|total_transaction_val|
+----+---------------------+
|2025|                25700|
|2027|                25700|
|2023|                28100|
|2022|                29800|
|2026|                25700|
|2029|                25700|
|2030|                 9500|
|2028|                25700|
|2024|                25700|
+----+---------------------+




<details>
    <summary>Click here for Solution</summary>

```python

# calculate the total transaction value for each year in df1.
total_value_per_year = df1.groupBy('year').agg(sum("transaction_amount").alias("total_transaction_val"))

# show the total transaction value for each year in df1.
total_value_per_year.show()

```

</details>


#### Task 15: Write the result to HDFS

Write `total_value_per_year` to HDFS in the CSV format to file named **total_value_per_year**.



In [29]:
#Write total_value_per_year to HDFS in the CSV format
# total_value_per_year.write \
#     .mode("overwrite") \
#     .csv("hdfs://localhost:9000/user/mohammad/total_value_per_year")

total_value_per_year.write \
    .mode("overwrite") \
    .csv("total_value_per_year")



<details>
    <summary>Click here for Solution</summary>

```python

#Write total_value_per_year to HDFS in the CSV format

total_value_per_year.write.mode("overwrite").csv("total_value_per_year.csv")

```

</details>


### Congratulations! You have completed the lab.
This practice project provides hands-on experience with data transformation and integration using PySpark. You've performed various tasks, including adding columns, renaming columns, dropping unnecessary columns, joining dataframes, and writing the results into both a Hive warehouse and an HDFS file system.


## Authors

Raghul Ramesh

Lavanya T S


<!--## Change Log -->


<!--|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-09-01|0.1|Lavanya T S|Initial version|
|2023-09-08|0.2|Pornima More|QA pass with edits|-->


<h3 align="center"> &#169; IBM Corporation. All rights reserved. <h3/>
